<a href="https://colab.research.google.com/github/ipeirotis-org/datasets/blob/main/notebooks/process_authors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-tasks

In [ ]:
import pandas as pd
from tqdm import tqdm

url = 'https://media.githubusercontent.com/media/sentian/SciImpactRanking/master/data/raw/authors.csv'

benchmark_df = pd.read_csv(url)

benchmark_df

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud import tasks_v2
from google.protobuf import timestamp_pb2
import datetime

# Create a client for Cloud Tasks
client = tasks_v2.CloudTasksClient()

# Your Google Cloud project ID and location
project = 'scholar-version2'
location = 'northamerica-northeast1'
queue = 'process-authors'

# Construct the fully qualified queue name
parent = client.queue_path(project, location, queue)

scholar_ids = benchmark_df['gs.id'].tolist()

for scholar_id in tqdm(scholar_ids):
    # The URL to your Cloud Function or Cloud Run service
    url = f'https://scholar.ipeirotis.org/api/author_stats/{scholar_id}'
    # Construct the request body
    task = {
        'http_request': {
            'http_method': tasks_v2.HttpMethod.GET,
            'url': url,
            'headers': {'Content-type': 'application/json'},
        }
    }
    # Add the task to the queue
    response = client.create_task(request={"parent": parent, "task": task})
    # print('Task created:', response.name)
